# Quantum Harmonic Oscillator Heat Engine
Optimize the output power of a heat engine based on a quantum harmonic oscillator (see section IVc of manuscript or [this](https://doi.org/10.1088/1367-2630/8/5/083) reference). The Hamiltonian of the system is:
\begin{equation}
	\hat{H}[u(t)] = \frac{1}{2m} \hat{p}^2 + \frac{1}{2}m (u(t)w_0)^2 \hat{q}^2,
\end{equation}
where $m$ is the mass of the system, $w_0$ is a fixed frequency and $\hat{p}$ and $\hat{q}$ are the momentum and position operators. The single continuous control parameter is $u(t)$. 
The coupling to the baths is described using the Lindblad master equation [see Eq. (9) of the manuscript]. The Lindblad operators and corresponding rates are gived by
\begin{align}
	\hat{A}^{(\alpha)}_{+,u(t)} &= \hat{a}_{u(t)}^\dagger, & \gamma^{(\alpha)}_{+,u(t)} &= \Gamma_\alpha \,n(\beta_\alpha u(t)\omega_0), \\
    \hat{A}^{(\alpha)}_{-,u(t)} &= \hat{a}_{u(t)}, & \gamma^{(\alpha)}_{-,u(t)} &= \Gamma_\alpha[1+ n(\beta_\alpha u(t) \omega_0 )],
\end{align}
where $\hat{a}_{u(t)}=(1/\sqrt{2})\sqrt{m\omega_0 u(t)}\,\hat{q} + i/\sqrt{m\omega_0 u(t)}\,\hat{p}$ and $\hat{a}_{u(t)}^\dagger$ are respectively the (control dependent) lowering and raising operators, $\Gamma_\alpha$ are constant rates, $n(x)=(\exp(x)-1)^{-1}$ is the Bose-Einstein distribution and $\beta_\alpha$ is the inverse temperature of bath $\alpha$.
#### Import modules

In [ ]:
import sys
import os
sys.path.append(os.path.join('..','src'))
import sac_tri
import sac_tri_envs

## Setup new Training
The following codes initiates a new training session. All training logs, parameters and saved states will be stored under the ```data``` folder, within a folder with the current date and time. 
- ```env_params``` is a dictionary with the environment parameters.
- ```training_hyperparams``` is a dictionary with training hyperparameters.
- ```log_info``` is a dictionary that specifices which quantities to log.

The parameters below were used to produce Fig. 5 of the manuscript.

In [ ]:
#LEFT PANEL PARAMETERS
wc = 1.
wh = 1.99324
LR = 0.001
ALPHA_START = 50.
ALPHA_DECAY = 24000
INITIAL_RANDOM_STEPS = 5000

#RIGHT PANEL PARAMETERS
# wc = 0.7
# wh = 3.
# LR = 0.0005
# ALPHA_START = 300.
# ALPHA_DECAY = 48000
# INITIAL_RANDOM_STEPS = 10000

#THE FOLLOWING PARAMETERS ARE SHARED BY BOTH PANELS
tc = 1./2.
th = 4.98309
w0 = 2.
min_u = wc/w0 
max_u = wh/w0 
dt = 0.2

env_params = {
    "g0": 0.6,                                    #\Gamma of bath 0
    "g1": 0.6,                                    #\Gamma of bath 1
    "b0": 1./th,                                  #inverse temperature \beta of bath 0
    "b1": 1./tc,                                  #inverse temperature \beta of bath 1
    "min_u": min_u,                               #minimum value of action u
    "max_u": max_u,                               #maximum value of action u
    "w0": w0,                                     #\omega_0
    "dt": dt,                                     #timestep \Delta t
    "reward_extra_coeff": 10.                     #the reward is multiplied by this factor
}  
training_hyperparams = {
    "BATCH_SIZE": 256,                            #batch size
    "LR": LR,                                     #learning rate
    "ALPHA_START": ALPHA_START,                   #initial value of SAC temperature
    "ALPHA_END": 0.,                              #final value of SAC temperature
    "ALPHA_DECAY": ALPHA_DECAY,                   #exponential decay of SAC temperature
    "REPLAY_MEMORY_SIZE": 192000,                 #size of replay buffer
    "POLYAK": 0.995,                              #polyak coefficient
    "LOG_STEPS": 6000,                            #save logs and display training every number of steps
    "GAMMA": 0.998,                               #RL discount factor
    "HIDDEN_SIZES": (256,256),                    #size of hidden layers 
    "SAVE_STATE_STEPS": 80000,                    #saves complete state of trainig every number of steps
    "INITIAL_RANDOM_STEPS": INITIAL_RANDOM_STEPS, #number of initial uniformly random steps
    "UPDATE_AFTER": 1000,                         #start minimizing loss function after initial steps
    "UPDATE_EVERY": 50,                           #performs this many updates every this many steps
    "USE_CUDA": False                             #use cuda for computation
}
log_info = {
    "log_running_reward": True,                   #log running reward 
    "log_running_loss": True,                     #log running loss
    "log_actions": True,                          #log chosen actions
    "extra_str": "_harmonic_engine"               #extra string to append to training folder
}

train = sac_tri.SacTrain()
train.initialize_new_train(sac_tri_envs.HarmonicEngineLogState, env_params, training_hyperparams, log_info)

#### Train
Perform a given number of training steps. It can be run multiple times.

In [ ]:
train.train(300000)

## Save the State
The full state of the training session is saved every ```SAVE_STATE_STEPS``` steps. Run this command if you wish to manually save the current state.

In [ ]:
train.save_full_state()

## Load Existing Training
Any training session that was saved can be loaded specifying the training session folder in ```log_dir```. This will produce a new folder for logging with the current date-time. The following loads the latest save in a folder named ```"2021_04_14-10_36_57_harmonic_engine"```.

In [ ]:
log_dir = "../data/2021_04_14-10_36_57_harmonic_engine/"
train = sac_tri.SacTrain()
train.load_train(log_dir)

#### Train
Perform a given number of training steps. It can be run multiple times.

In [ ]:
train.train(50000)

## Manually changing the learning rate
The following is NOT RECOMMENDED, since the change is not logged. However, it is possible to change the learning rate during training. The following changes the learning rate to 0.0001.

In [ ]:
for g in train.pi_optimizer.param_groups:
    g['lr'] = 0.0001
for g in train.q_optimizer.param_groups:
    g['lr'] = 0.0001